In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt 
import seaborn as sns # visualization tool
from pandas.io.json import json_normalize
import matplotlib.pyplot as plot
import re
def country_dataset(csv_file, json_file):
    df = pd.read_csv(csv_file)
    #df.isnull().sum()
    with open(json_file) as json_data:
        data = json.load(json_data)
    df_json = pd.DataFrame.from_dict(json_normalize(data), orient='columns')  #df is created for json real_state.json
    category_list = []
    for i in df_json["items"]:
        for j in i:
            category_list.append(j)
    pattern_id = r"'id': '(.*?)'"
    pattern_title = r"'title': '(.*?)'"
    id_list = []
    title_list = []
    for each in category_list:
        if re.search(pattern_id, str(each)):
            id = re.findall(pattern_id, str(each))
            id_list.append(int(id[0]))
        if re.search(pattern_title, str(each)):
            title = re.findall(pattern_title, str(each))
            title_list.append(title[0])
    dict_category = dict(zip(id_list, title_list))
    dict_category
    df_category = pd.DataFrame()
    df_category["category_id"] = id_list
    df_category["category_title"] = title_list
    #df.info()
    rows = len(df['trending_date'])
    trending_date_format = []
    clean_dt_format_1 = []  #list is created to store temp date format
    for i in range(rows):
        year = df.trending_date[i][0:2]
        date = df.trending_date[i][3:5]
        month = df.trending_date[i][6:8]
        dt_format = date + "-" + month + "-" + "20" + year
        trending_date_format.append(dt_format)
    df['trending_dt'] = trending_date_format
    rows = len(df['publish_time'])
    publish_dt = []
    publish_time = []
    for i in range(rows):
        date_time = df.publish_time[i].split("T")
        year = date_time[0][0:4]
        month = date_time[0][5:7]
        date = date_time[0][8:10]
        dt_format = date + "-" + month + "-" + year
        publish_dt.append(dt_format)
        time_format = date_time[1][0:5]
        publish_time.append(time_format)

    df["publish_date"] = publish_dt
    df["publish_time"] = publish_time
    df_category
    df_merged = pd.merge(df, df_category, on=['category_id'], how='inner')
    #df_merged.head()
    df_YT = pd.DataFrame()
    df_YT["trending_date"] = df_merged["trending_dt"]
    df_YT["video_id"] = df_merged["video_id"]
    df_YT["title"] = df_merged["title"]
    df_YT["channel_title"] = df_merged["channel_title"]
    df_YT["publish_date"] = df_merged["publish_date"]
    df_YT["publish_time"] = df_merged["publish_time"]
    df_YT["views"] = df_merged["views"]
    df_YT["likes"] = df_merged["likes"]
    df_YT["dislikes"] = df_merged["dislikes"]
    df_YT["comment_count"] = df_merged["comment_count"]
    df_YT["category_id"] = df_merged["category_id"]
    df_YT["category_title"] = df_merged["category_title"]
    df_YT['trending_date'] = pd.to_datetime(df_YT['trending_date'])
    df_YT['publish_date'] = pd.to_datetime(df_YT['publish_date'])
    #df_YT = df_YT.sort_values(['trending_date','views'],ascending=[False, False])
    def dollars_earned(views):
        per_view_dollar = 0
        no_of_views = views
        if csv_file == 'USvideos.csv':   
            per_view_dollar = 7.60
        elif csv_file == 'INvideos.csv':
            per_view_dollar = 7.60
        elif csv_file == 'GBvideos.csv':
            per_view_dollar = 7.60
        dollar_earned = (no_of_views/1000)*per_view_dollar
        return dollar_earned
    rows = len(df_YT['views'])
    dollar_earned_list = []
    for each in range(rows):
        dollars = dollars_earned(df_YT.views[each])
        dollar_earned_list.append(round(dollars,2))  
    df_YT['dollars_earned'] = dollar_earned_list
    df_YT = df_YT.sort_values(['trending_date','views'],ascending=[False, False])
    if csv_file == 'USvideos.csv':        
        lat = 39.022774
        long = -76.921753
        country_code = "US"
    elif csv_file == 'INvideos.csv':
        lat = 18.516726
        long = 73.856255
        country_code = "India"
    elif csv_file == 'GBvideos.csv':
        lat = 51.509865
        long = -0.118092
        country_code = "UK"
    df_YT.insert(13, "country", country_code)
    df_YT.insert(14, "lat", lat)
    df_YT.insert(15, "long", long)
    return df_YT

In [2]:
csv_file = ['USvideos.csv', 'INvideos.csv', 'GBvideos.csv']
json_file = ['US_category_id.json', 'IN_category_id.json', 'GB_category_id.json']
#country_dataset(csv_file, json_file)

In [3]:
for i in range(len(csv_file)):
    df_YT1 = pd.DataFrame()
    df_YT1 = country_dataset(csv_file[i], json_file[i])
    if csv_file[i] == 'USvideos.csv': 
        df_US = pd.DataFrame()
        df_US = df_YT1
    elif csv_file[i] == 'INvideos.csv':
        df_IN = pd.DataFrame()
        df_IN = df_YT1
    elif csv_file[i] == 'GBvideos.csv':
        df_UK = pd.DataFrame()
        df_UK = df_YT1
df_YT_countries = pd.concat([df_US, df_IN, df_UK])
df_YT_countries.to_csv("YT_solution.csv", index = False)

In [4]:
#df_YT_countries = pd.concat([df_lin, df_null])